<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Mum_Formasyonlar%C4%B1_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Zaman dilimi seçenekleri:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Zaman dilimini seçin (1-9): ")
    return intervals.get(choice, ('Günlük', Interval.in_daily))

# Mum analizi ve formasyonları tespit etme fonksiyonları
def detect_patterns(data):
    df = data.copy()

    # Boğa Yutan (Bullish Engulfing)
    df['Boga_Yutan'] = (
        (df['Close'] > df['Open']) &
        (df['Open'] < df['Close'].shift(1)) &
        (df['Close'] > df['Open'].shift(1))
    )

    # Ayı Yutan (Bearish Engulfing)
    df['Ayi_Yutan'] = (
        (df['Close'] < df['Open']) &
        (df['Open'] > df['Close'].shift(1)) &
        (df['Close'] < df['Open'].shift(1))
    )

    # Çekiç (Hammer)
    df['Cekic'] = (
        (df['Close'] > df['Open']) &
        ((df['Open'] - df['Low']) > 2 * (df['Close'] - df['Open'])) &
        ((df['High'] - df['Close']) < (df['Close'] - df['Open']))
    )

    # Ters Çekiç (Inverted Hammer)
    df['Ters_Cekic'] = (
        (df['Close'] > df['Open']) &
        ((df['High'] - df['Close']) > 2 * (df['Close'] - df['Open'])) &
        ((df['Open'] - df['Low']) < (df['Close'] - df['Open']))
    )

    # Doji
    df['Doji'] = abs(df['Close'] - df['Open']) < (df['High'] - df['Low']) * 0.1

    # Sabah Yıldızı (Morning Star)
    df['Sabah_Yildizi'] = (
        (df['Close'].shift(2) < df['Open'].shift(2)) &  # İlk mum ayı
        (abs(df['Close'].shift(1) - df['Open'].shift(1)) < (df['High'].shift(1) - df['Low'].shift(1)) * 0.3) &  # İkinci mum küçük
        (df['Close'] > df['Open']) &  # Üçüncü mum boğa
        (df['Close'] > (df['Open'].shift(2) + df['Close'].shift(2)) / 2)  # Üçüncü mum birinci mumun ortasını aşmalı
    )

    # Karanlık Bulut Örtüsü (Dark Cloud Cover)
    df['Karanlik_Bulut'] = (
        (df['Close'] < df['Open']) &
        (df['Open'] > df['Close'].shift(1)) &
        (df['Close'] < (df['Open'].shift(1) + df['Close'].shift(1)) / 2)
    )

    # Üç Beyaz Asker (Three White Soldiers)
    df['Uc_Beyaz_Asker'] = (
        (df['Close'] > df['Open']) &
        (df['Close'].shift(1) > df['Open'].shift(1)) &
        (df['Close'].shift(2) > df['Open'].shift(2)) &
        (df['Close'] > df['Close'].shift(1)) &
        (df['Close'].shift(1) > df['Close'].shift(2))
    )

    # Üç Siyah Karga (Three Black Crows)
    df['Uc_Siyah_Karga'] = (
        (df['Close'] < df['Open']) &
        (df['Close'].shift(1) < df['Open'].shift(1)) &
        (df['Close'].shift(2) < df['Open'].shift(2)) &
        (df['Close'] < df['Close'].shift(1)) &
        (df['Close'].shift(1) < df['Close'].shift(2))
    )

    # Bullish Harami
    df['Bullish_Harami'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # İlk mum ayı
        (df['Close'] > df['Open']) &  # İkinci mum boğa
        (df['Open'] > df['Close'].shift(1)) &  # İkinci mumun açılışı birinci mumun kapanışından büyük
        (df['Close'] < df['Open'].shift(1))  # İkinci mumun kapanışı birinci mumun açılışından küçük
    )

    # Bearish Harami
    df['Bearish_Harami'] = (
        (df['Close'].shift(1) > df['Open'].shift(1)) &  # İlk mum boğa
        (df['Close'] < df['Open']) &  # İkinci mum ayı
        (df['Open'] < df['Close'].shift(1)) &  # İkinci mumun açılışı birinci mumun kapanışından küçük
        (df['Close'] > df['Open'].shift(1))  # İkinci mumun kapanışı birinci mumun açılışından büyük
    )

    # Hamile Boğa (Bullish Harami)
    df['Hamile_Boga'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # İlk mum bearish
        (df['Close'] > df['Open']) &  # İkinci mum bullish
        (df['Open'] > df['Close'].shift(1)) &  # İkinci mumun açılışı birinci mumun kapanışından büyük
        (df['Close'] < df['Open'].shift(1))  # İkinci mumun kapanışı birinci mumun açılışından küçük
    )

    # Güvercin Yuvası (Homing Pigeon)
    df['Guvercin_Yuvasi'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # İlk mum bearish
        (df['Close'] < df['Open']) &  # İkinci mum bearish
        (df['Open'] < df['Open'].shift(1)) &  # İkinci mumun açılışı birinci mumun açılışından küçük
        (df['Close'] > df['Close'].shift(1))  # İkinci mumun kapanışı birinci mumun kapanışından büyük
    )


    # Piercing Pattern (Delici Mum)
    df['Delici_Mum'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # İlk mum bearish
        (df['Close'] > df['Open']) &  # İkinci mum bullish
        (df['Close'] > (df['Open'].shift(1) + df['Close'].shift(1)) / 2)  # İkinci mum, birinci mumun gövdesinin %50'sini aşmalı
    )

    # Bullish Marubozu
    df['Bullish_Marubozu'] = (
        (df['Close'] > df['Open']) &  # Bullish
        ((df['Close'] - df['Open']) > (df['High'] - df['Low']) * 0.8) &  # Gövde toplam uzunluğun %80'inden büyük
        ((df['High'] - df['Close']) < (df['Close'] - df['Open']) * 0.1) &  # Üst fitil kısa
        ((df['Open'] - df['Low']) < (df['Close'] - df['Open']) * 0.1)  # Alt fitil kısa
    )

    # Bearish Marubozu
    df['Bearish_Marubozu'] = (
        (df['Close'] < df['Open']) &  # Bearish
        ((df['Open'] - df['Close']) > (df['High'] - df['Low']) * 0.8) &  # Gövde toplam uzunluğun %80'inden büyük
        ((df['High'] - df['Open']) < (df['Open'] - df['Close']) * 0.1) &  # Üst fitil kısa
        ((df['Close'] - df['Low']) < (df['Open'] - df['Close']) * 0.1)  # Alt fitil kısa
    )

    # Bullish Harami Cross
    df['Bullish_Harami_Cross'] = (
        (df['Close'].shift(1) < df['Open'].shift(1)) &  # İlk mum ayı
        (df['Doji'].shift(0)) &  # İkinci mum bir Doji
        (df['Close'] < df['Open'].shift(1)) &  # Doji, birinci mumun gövdesinin içinde
        (df['Open'] > df['Close'].shift(1))
    )

    # Bearish Harami Cross
    df['Bearish_Harami_Cross'] = (
        (df['Close'].shift(1) > df['Open'].shift(1)) &  # İlk mum boğa
        (df['Doji'].shift(0)) &  # İkinci mum bir Doji
        (df['Close'] > df['Open'].shift(1)) &  # Doji, birinci mumun gövdesinin içinde
        (df['Open'] < df['Close'].shift(1))
    )

    #Evening_Star
    df['Aksam_Yildizi'] = (
        (df['Close'].shift(2) > df['Open'].shift(2)) &  # İlk mum boğa
        (abs(df['Close'].shift(1) - df['Open'].shift(1)) < (df['High'].shift(1) - df['Low'].shift(1)) * 0.3) &  # İkinci mum küçük gövde
        (df['Close'] < df['Open']) &  # Üçüncü mum ayı
        (df['Close'] < (df['Open'].shift(2) + df['Close'].shift(2)) / 2)  # Üçüncü mum birinci mumun gövdesinin yarısına inmiş
    )

    #Tweezers Bottom (Cımbız Dibi)
    df['Tweezers_Bottom'] = (
        (df['Low'] == df['Low'].shift(1)) &  # İki mumun alt fitili eşit
        (df['Close'] > df['Open']) &  # İkinci mum boğa
        (df['Close'].shift(1) < df['Open'].shift(1))  # İlk mum ayı
    )

    return df

# Ana fonksiyon
def main():
    tv = TvDatafeed()

    # Kullanıcıdan zaman dilimi seçimi
    interval_name, interval = get_interval_choice()

    # Türkiye pazarındaki hisseleri al
    hisseler = get_all_symbols(market='turkey')
    hisseler = [symbol.replace('BIST:', '') for symbol in hisseler]
    hisseler = sorted(hisseler)

    # Formasyonların kategorisi
    formasyon_kategorisi = {
        'Boga_Yutan': 'Boğa',
        'Ayi_Yutan': 'Ayı',
        'Cekic': 'Boğa',
        'Ters_Cekic': 'Boğa',
        'Doji': 'Nötr',
        'Sabah_Yildizi': 'Boğa',
        'Karanlik_Bulut': 'Ayı',
        'Uc_Beyaz_Asker': 'Boğa',
        'Uc_Siyah_Karga': 'Ayı',
        'Bullish_Harami': 'Boğa',
        'Bearish_Harami': 'Ayı',
        'Delici_Mum': 'Boğa',
        'Bearish_Marubozu': 'Ayı',
        'Bullish_Marubozu': 'Boğa',
        'Bearish_Harami_Cross': 'Ayı',
        'Bullish_Harami_Cross': 'Boğa',
        'Aksam_Yildizi': 'Ayı',
        'Hamile_Boga': 'Boğa',
        'Tweezers_Bottom': 'Boğa',
        'Guvercin_Yuvasi': 'Boğa'

    }

    # Sonuçları saklamak için DataFrame oluşturma
    df_signals = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Formasyon Türü', 'Kategori', 'Zaman Dilimi', 'Tarih'])

    for hisse in tqdm(hisseler, desc="Hisseler Taraniyor"):
        try:
            # Veriyi al
            data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval, n_bars=6)

            if not isinstance(data, pd.DataFrame):
                continue  # Eğer veri DataFrame değilse atla

            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()

            # Formasyon analizi yap
            analyzed_data = detect_patterns(data)

            # Formasyonları kaydet
            for pattern in formasyon_kategorisi.keys():
                last_row = analyzed_data.tail(1)
                detected = last_row[last_row[pattern]]
                for _, row in detected.iterrows():
                    df_signals.loc[len(df_signals)] = [
                        hisse, row['Close'], pattern, formasyon_kategorisi[pattern], interval_name, row['datetime']
                    ]

        except Exception as e:
            print(f"{hisse} işlenirken hata oluştu: {e}")

    # Sonuçları Excel'e kaydet
    excel_file = 'mum_formasyonlari.xlsx'
    with pd.ExcelWriter(excel_file) as writer:
        df_signals.to_excel(writer, sheet_name='Mum Formasyonları', index=False)

    print(f"Sonuçlar '{excel_file}' dosyasına kaydedildi.")

if __name__ == "__main__":
    main()




  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-pfsgmaav
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-pfsgmaav
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Zaman dilimi seçenekleri:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Zaman dilimini seçin (1-9): 7


Hisseler Taraniyor: 100%|██████████| 585/585 [05:45<00:00,  1.69it/s]


Sonuçlar 'mum_formasyonlari.xlsx' dosyasına kaydedildi.
